In [1]:
%pip install langchain langchain-openai python-dotenv

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 23.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 26.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 23.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/720.5 kB ? eta -:--:--
   --------------------------------------- 720.5/720.5 kB 14.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/894.0 kB ? eta -:--:--
   --------------------------------------- 894.0/894.0 kB 10.2 MB/s eta 0:00:00

   - --------------------------------------  1/37 [urllib3]
   - --------------------------------------  1/37 [urllib3]
   --- ------------------------------------  3/37 [tqdm]
   ---- -----------------------------------  4/37 [tenacity]
   ------ -----------

In [14]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [17]:
# -------------------------------
# ✅ 세션 히스토리 저장소
# -------------------------------
store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [ ]:
# -------------------------------
# ✅ 기본 구성 요소
# -------------------------------
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)
parser = StrOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 한국어 챗봇입니다. 사용자 질문에 친절하고 자세히 응답해주세요."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# 체인 구성
chain = prompt | llm | parser

# 세션 기반 챗봇
chatbot = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [19]:
# -------------------------------
# ✅ 테스트 실행
# -------------------------------
response = chatbot.invoke(
    {"input": "안녕하세요. 자기소개 해주세요."},
    config={"configurable": {"session_id": "test_user"}}
)

print("\n[test_user] 챗봇 응답:\n", response)


[test_user] 챗봇 응답:
 안녕하세요! 저는 여러분의 질문에 답하고, 다양한 정보를 제공하기 위해 만들어진 AI 챗봇입니다. 한국어로 소통하며, 여러분이 궁금한 점이나 필요한 정보에 대해 친절하게 도와드릴 수 있습니다. 어떤 질문이든지 편하게 해주세요!


In [23]:
response = chatbot.invoke(
    {"input": "내 전 질문이 뭐지?"},
    config={"configurable": {"session_id": "test_user"}}
)

print("\n[test_user] 챗봇 응답:\n", response)


[test_user] 챗봇 응답:
 당신의 전 질문은 "내 전 질문이 뭐지?"입니다. 이 질문에 대해 답변을 드리고 있습니다. 더 궁금한 점이나 다른 질문이 있으시면 언제든지 말씀해 주세요!


In [25]:
# 검색된 의학지싟을 넣을 프롬프트 작성
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 한국어 챗봇입니다. 사용자 질문에 친절하고, 반드시 제공된 문서 내용을 기반으로 답변해주세요."),
    ("system", "문서 내용:\n{context}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [26]:
# 2. 문서 내용을 임시로 문자열로 넣기 (파일 로딩 예제는 아래에 있음)
doc_content = "나는 의학지식인데 의학지식테스트야 앞으로 해당 전용 llm의 응답을 넣어서 너가 대답하게 될거야"


In [27]:
# 4. LLM & 파서 & 체인 구성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
parser = StrOutputParser()
chain = prompt | llm | parser

In [ ]:
chatbot = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 6. 질문 수행(문서 유사도도 판단해야될듯 streamlit 질문입력 -> 카테고리 분석(문서유사도0.5이상만 카테고리 분류))
# (문서 유사도 0.5미만일시 context안넣고 바로 답변)
# (문서 유사도 0.5이상일시 해당문서 퓨샷하여 카테고리분류)->분류된 카테고리에 맵핑된 llm에 질문 전달(프록시패턴으로 라우팅하기)->해당응답 context에 담아 답변하게 하기
# (우리플젝 일단끝)
# (내일 코드 분리하고 프록시패턴으로 해당 카테고리에 질문 응답 추상화하기)
# (streamlit 까지 작업 꼭하기)
response = chatbot.invoke(
    {"input": "너는 뭐지?", "context": doc_content},
    config={"configurable": {"session_id": "doc_user"}}
)

print("\n[doc_user] 챗봇 응답:\n", response)


[doc_user] 챗봇 응답:
 저는 의학 지식에 기반한 챗봇입니다. 의학 관련 질문에 대한 정보를 제공하고, 여러분의 궁금증을 해결하는 데 도움을 드립니다. 어떤 질문이든지 해주세요!
